In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from flask import Flask, request, jsonify


# Membaca data dari CSV
data = pd.read_csv('Data_Camera.csv')

# Menampilkan beberapa baris pertama untuk verifikasi
print("Data awal:")
print(data.head())

Data awal:
                 Nama Kamera Brand Kamera Jenis Sensor  Color Depth  \
0             Canon EOS 300D        Canon        APS-C         21.0   
1              Canon EOS 10D        Canon        APS-C         21.0   
2         Nikon Coolpix P340        Nikon       1/1.7"         20.7   
3         Nikon Coolpix P330        Nikon       1/1.7"         21.0   
4  Panasonic Lumix DMC FX150    Panasonic       1/1.7"         18.4   

   Dynamic Range  LowLight ISO      Price   Preferensi  
0           10.8           544  5,500,000  Profesional  
1           10.9           571  1,200,000         Hobi  
2           11.9           273  2,838,000       Pemula  
3           11.7           213  1,150,000       Pemula  
4            9.6           101  3,100,000         Hobi  


In [2]:
# Menghapus tanda pemisah ribuan dan mengubah tipe data menjadi float
if 'Price' in data.columns:
    data['Price'] = data['Price'].str.replace(',', '').astype(float)
else:
    raise ValueError("Kolom 'Price' tidak ditemukan dalam data.")

# Menampilkan kolom Price untuk verifikasi
print("\nKolom Price setelah konversi:")
print(data['Price'].head())

# Pastikan kolom lain yang diperlukan juga dalam format numerik, kecuali 'Jenis Sensor'
columns_to_convert = ['Color Depth', 'LowLight ISO']
for col in columns_to_convert:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    else:
        raise ValueError(f"Kolom '{col}' tidak ditemukan dalam data.")

# Menampilkan data setelah konversi untuk verifikasi
print("\nData setelah konversi ke numerik:")
print(data[columns_to_convert].head())

# Menghapus baris yang memiliki nilai NaN pada kolom yang telah dikonversi
data.dropna(subset=['Price', 'Color Depth', 'LowLight ISO'], inplace=True)

# Menampilkan data setelah menghapus baris dengan nilai NaN
print("\nData setelah menghapus NaN:")
print(data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']].head())

# Normalisasi data, kecuali 'Jenis Sensor'
if not data.empty:
    # Normalisasi data
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data[['Price', 'Color Depth', 'LowLight ISO']])

    # Memasukkan kembali data yang telah dinormalisasi ke dalam DataFrame
    data[['Price', 'Color Depth', 'LowLight ISO']] = data_scaled

    # Menampilkan data untuk memverifikasi hasil
    print("\nData setelah normalisasi:")
    print(data.head())
else:
    print("\nTidak ada data yang tersisa setelah pembersihan. Silakan periksa data input Anda.")
    exit()  # Keluar dari program jika tidak ada data yang tersisa

# Label encoding untuk 'Jenis Sensor'
label_encoder = LabelEncoder()
label_encoder.fit(data['Jenis Sensor'])

# Encode the 'Jenis Sensor' column
data['Jenis Sensor'] = label_encoder.transform(data['Jenis Sensor'])

# Menampilkan data untuk memverifikasi hasil
print("\nData setelah encoding:")
print(data.head())

# Tentukan kriteria utama
kriteria = ['Price', 'Jenis Sensor', 'Color Depth', 'Dynamic Range', 'LowLight ISO']


Kolom Price setelah konversi:
0    5500000.0
1    1200000.0
2    2838000.0
3    1150000.0
4    3100000.0
Name: Price, dtype: float64

Data setelah konversi ke numerik:
   Color Depth  LowLight ISO
0         21.0           544
1         21.0           571
2         20.7           273
3         21.0           213
4         18.4           101

Data setelah menghapus NaN:
       Price Jenis Sensor  Color Depth  LowLight ISO
0  5500000.0        APS-C         21.0           544
1  1200000.0        APS-C         21.0           571
2  2838000.0       1/1.7"         20.7           273
3  1150000.0       1/1.7"         21.0           213
4  3100000.0       1/1.7"         18.4           101

Data setelah normalisasi:
                 Nama Kamera Brand Kamera Jenis Sensor  Color Depth  \
0             Canon EOS 300D        Canon        APS-C     0.525424   
1              Canon EOS 10D        Canon        APS-C     0.525424   
2         Nikon Coolpix P340        Nikon       1/1.7"     0.474576   

In [3]:
# Memastikan bahwa ada data yang tersisa untuk dinormalisasi
if not data.empty:
    # Normalisasi data
    scaler = MinMaxScaler()
    data[['Price', 'Color Depth', 'LowLight ISO']] = scaler.fit_transform(data[['Price', 'Color Depth', 'LowLight ISO']])

    # Encoding 'Jenis Sensor' menggunakan LabelEncoder atau OneHotEncoder
    label_encoder = LabelEncoder()
    data['Jenis Sensor'] = label_encoder.fit_transform(data['Jenis Sensor'])

    # Menampilkan data untuk memverifikasi hasil
    print("\nData setelah normalisasi dan encoding:")
    print(data.head())
else:
    print("\nTidak ada data yang tersisa setelah pembersihan. Silakan periksa data input Anda.")
    exit()  # Keluar dari program jika tidak ada data yang tersisa


Data setelah normalisasi dan encoding:
                 Nama Kamera Brand Kamera  Jenis Sensor  Color Depth  \
0             Canon EOS 300D        Canon             5     0.525424   
1              Canon EOS 10D        Canon             5     0.525424   
2         Nikon Coolpix P340        Nikon             1     0.474576   
3         Nikon Coolpix P330        Nikon             1     0.525424   
4  Panasonic Lumix DMC FX150    Panasonic             1     0.084746   

   Dynamic Range  LowLight ISO     Price   Preferensi  
0           10.8      0.372166  0.528169  Profesional  
1           10.9      0.393276  0.023474         Hobi  
2           11.9      0.160281  0.215728       Pemula  
3           11.7      0.113370  0.017606       Pemula  
4            9.6      0.025801  0.246479         Hobi  


In [4]:
# Tentukan kriteria utama
kriteria = ['Price', 'Jenis Sensor', 'Color Depth', 'Dynamic Range', 'LowLight ISO']

# Matriks perbandingan berpasangan untuk kriteria utama
pairwise_matrix = np.array([
    [1, 3, 5, 7, 9],
    [1/3, 1, 3, 5, 7],
    [1/5, 1/3, 1, 3, 5],
    [1/7, 1/5, 1/3, 1, 3],
    [1/9, 1/7, 1/5, 1/3, 1]
])

# Fungsi untuk menghitung bobot dari matriks perbandingan berpasangan
def calculate_weights(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)].real
    weights = max_eigenvector / np.sum(max_eigenvector)
    return weights

# Hitung bobot kriteria utama
kriteria_weights = calculate_weights(pairwise_matrix)
print("Bobot Kriteria Utama:", dict(zip(kriteria, kriteria_weights)))

# Jika kita memiliki subkriteria, kita juga perlu membuat matriks perbandingan berpasangan untuk subkriteria setiap kriteria.
# Misalnya untuk subkriteria Harga:
subkriteria_harga = ['< 500.000', '500.000-1.000.000', '> 1.000.000']
pairwise_matrix_harga = np.array([
    [1, 2, 3],
    [1/2, 1, 2],
    [1/3, 1/2, 1]
])

# Hitung bobot subkriteria Harga
subkriteria_harga_weights = calculate_weights(pairwise_matrix_harga)
print("Bobot Subkriteria Harga:", dict(zip(subkriteria_harga, subkriteria_harga_weights)))

# Definisikan bilangan fuzzy segitiga
fuzzy_scale = {
    1: (1, 1, 1),
    3: (2, 3, 4),
    5: (4, 5, 6),
    7: (6, 7, 8),
    9: (8, 9, 10)
}

Bobot Kriteria Utama: {'Price': 0.5128128127814086, 'Jenis Sensor': 0.2614990557217761, 'Color Depth': 0.12897642312026958, 'Dynamic Range': 0.06337652767438601, 'LowLight ISO': 0.033335180702159815}
Bobot Subkriteria Harga: {'< 500.000': 0.5396145502210747, '500.000-1.000.000': 0.29696133121249724, '> 1.000.000': 0.16342411856642797}


In [5]:
# Matriks perbandingan berpasangan fuzzy (contoh)
fuzzy_pairwise_matrix = [
    [(1, 1, 1), (2, 3, 4), (4, 5, 6)],
    [(1/4, 1/3, 1/2), (1, 1, 1), (2, 3, 4)],
    [(1/6, 1/5, 1/4), (1/4, 1/3, 1/2), (1, 1, 1)]
]

# Convert to numpy array for easier manipulation
fuzzy_pairwise_matrix = np.array(fuzzy_pairwise_matrix, dtype=object)

# Function to perform fuzzy multiplication
def fuzzy_multiply(a, b):
    return (a[0] * b[0], a[1] * b[1], a[2] * b[2])

# Function to perform fuzzy addition
def fuzzy_add(a, b):
    return (a[0] + b[0], a[1] + b[1], a[2] + b[2])

# Compute fuzzy sum for each row
fuzzy_sums = [reduce(fuzzy_add, row) for row in fuzzy_pairwise_matrix]

# Normalize the fuzzy sums
total_sum = reduce(fuzzy_add, fuzzy_sums)

# Normalize each fuzzy sum
def normalize_fuzzy(sums, total_sum):
    return (sums[0] / total_sum[2], sums[1] / total_sum[1], sums[2] / total_sum[0])

fuzzy_extent = [normalize_fuzzy(sums, total_sum) for sums in fuzzy_sums]

# Defuzzify fuzzy weights using centroid method
def defuzzify(triangular_fuzzy_number):
    l, m, u = triangular_fuzzy_number
    return (l + m + u) / 3

# Defuzzify fuzzy weights
crisp_weights = [defuzzify(weight) for weight in fuzzy_extent]

print("Crisp Weights:")
print(crisp_weights)

Crisp Weights:
[0.6439333175373473, 0.31366352794573205, 0.11025486140972351]


In [6]:
# Periksa apakah data kosong
if data.empty:
    print("Dataset kosong.")
else:
    # Periksa apakah kolom yang diperlukan ada
    required_columns = ['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO', 'Preferensi']
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        print(f"Kolom yang hilang: {missing_columns}")
        exit()  # Keluar dari program jika ada kolom yang hilang

    # Periksa nilai kosong
    null_values = data[required_columns].isnull().sum()
    print(f"Nilai kosong dalam kolom:\n{null_values}")

    # Periksa beberapa baris pertama dari DataFrame
    print(data.head())

Nilai kosong dalam kolom:
Price           0
Jenis Sensor    0
Color Depth     0
LowLight ISO    0
Preferensi      0
dtype: int64
                 Nama Kamera Brand Kamera  Jenis Sensor  Color Depth  \
0             Canon EOS 300D        Canon             5     0.525424   
1              Canon EOS 10D        Canon             5     0.525424   
2         Nikon Coolpix P340        Nikon             1     0.474576   
3         Nikon Coolpix P330        Nikon             1     0.525424   
4  Panasonic Lumix DMC FX150    Panasonic             1     0.084746   

   Dynamic Range  LowLight ISO     Price   Preferensi  
0           10.8      0.372166  0.528169  Profesional  
1           10.9      0.393276  0.023474         Hobi  
2           11.9      0.160281  0.215728       Pemula  
3           11.7      0.113370  0.017606       Pemula  
4            9.6      0.025801  0.246479         Hobi  


In [7]:
# Lanjutkan dengan train_test_split jika semua pemeriksaan lolos
if not data.empty and not missing_columns and null_values.sum() == 0:
    X = data[['Price', 'Jenis Sensor', 'Color Depth', 'LowLight ISO']]
    y = data['Preferensi']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # Latih model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
else:
    print("Data tidak cocok untuk dibagi dan dilatih.")

In [8]:
# 1. Evaluasi Model
# Setelah melatih model dengan data latih, langkah berikutnya adalah mengevaluasi performa model menggunakan data uji.

# Prediksi menggunakan data uji
y_pred = model.predict(X_test)

# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi model: {accuracy:.2f}")

# Laporan klasifikasi
report = classification_report(y_test, y_pred)
print("Laporan klasifikasi:\n", report)

Akurasi model: 0.68
Laporan klasifikasi:
               precision    recall  f1-score   support

        Hobi       0.50      0.40      0.44         5
      Pemula       0.57      0.80      0.67         5
 Profesional       0.88      0.78      0.82         9

    accuracy                           0.68        19
   macro avg       0.65      0.66      0.64        19
weighted avg       0.70      0.68      0.68        19



In [9]:
# # 2. Membuat Rekomendasi
# # Buat fungsi untuk memberikan rekomendasi kamera berdasarkan bobot kriteria yang dihitung dari F-AHP.

# # Fungsi untuk merekomendasikan kamera
# def rekomendasi_kamera(data, model, input_features):
#     # Normalisasi fitur input
#     scaler = MinMaxScaler()
#     input_features_scaled = scaler.fit_transform(input_features)

#     # Memprediksi preferensi menggunakan model yang telah dilatih
#     preferensi_prediksi = model.predict(input_features_scaled)

#     # Menggabungkan fitur input dengan hasil prediksi
#     hasil_rekomendasi = data.copy()
#     hasil_rekomendasi['Preferensi'] = preferensi_prediksi

#     # Mengurutkan berdasarkan preferensi tertinggi
#     hasil_rekomendasi = hasil_rekomendasi.sort_values(by='Preferensi', ascending=False)

#     return hasil_rekomendasi

# # Fitur input baru untuk rekomendasi
# input_features_baru = pd.DataFrame({
#     'Price': [1000000, 750000],
#     'Jenis Sensor': ['APS-C', 'Full Frame'],
#     'Color Depth': [24, 30],
#     'LowLight ISO': [3200, 6400]
# })

# # Periksa nilai 'Jenis Sensor' yang tidak dikenal
# unknown_values = set(input_features_baru['Jenis Sensor']) - set(label_encoder.classes_)
# if unknown_values:
#     print("Nilai 'Jenis Sensor' yang tidak dikenal dalam fitur input:", unknown_values)
# else:
#     input_features_baru['Jenis Sensor'] = label_encoder.transform(input_features_baru['Jenis Sensor'])

#     # Dapatkan rekomendasi
#     hasil_rekomendasi = rekomendasi_kamera(data, model, input_features_baru)
#     print("Rekomendasi kamera:")
#     print(hasil_rekomendasi.head())

In [10]:
# 2. Membuat Rekomendasi
# Buat fungsi untuk memberikan rekomendasi kamera berdasarkan bobot kriteria yang dihitung dari F-AHP.

# Fungsi untuk merekomendasikan kamera
def rekomendasi_kamera(data, model, input_features):
    # Normalisasi fitur input
    scaler = MinMaxScaler()
    input_features_scaled = scaler.fit_transform(input_features)

    # Memprediksi preferensi menggunakan model yang telah dilatih
    preferensi_prediksi = model.predict(input_features_scaled)

    # Menggabungkan fitur input dengan hasil prediksi
    hasil_rekomendasi = data.copy()
    hasil_rekomendasi['Preferensi'] = preferensi_prediksi

    # Mengurutkan berdasarkan preferensi tertinggi
    hasil_rekomendasi = hasil_rekomendasi.sort_values(by='Preferensi', ascending=False)

    return hasil_rekomendasi

# Fitur input baru untuk rekomendasi
input_features_baru = pd.DataFrame({
    'Price': [1000000, 750000],
    'Jenis Sensor': ['APS-C', 'Full Frame'],
    'Color Depth': [24, 30],
    'LowLight ISO': [3200, 6400]
})

# Buat objek OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)

# Reshape nilai kolom 'Jenis Sensor' menjadi bentuk array 2D
jenis_sensor_values = input_features_baru['Jenis Sensor'].values.reshape(-1, 1)

# Lakukan One-Hot Encoding pada kolom 'Jenis Sensor'
jenis_sensor_encoded = onehot_encoder.fit_transform(jenis_sensor_values)

# Dapatkan nama fitur dari OneHotEncoder
encoded_categories = onehot_encoder.categories_
jenis_sensor_feature_names = np.concatenate(encoded_categories).ravel()

# Buat DataFrame dari hasil One-Hot Encoding
jenis_sensor_encoded_df = pd.DataFrame(jenis_sensor_encoded, columns=jenis_sensor_feature_names)

# Gabungkan DataFrame hasil One-Hot Encoding dengan DataFrame input_features_baru
input_features_baru_encoded = pd.concat([input_features_baru.drop(columns=['Jenis Sensor']), jenis_sensor_encoded_df], axis=1)

# Periksa hasil One-Hot Encoding
print("Hasil One-Hot Encoding:")
print(input_features_baru_encoded)

# Periksa nilai 'Jenis Sensor' yang tidak dikenal setelah One-Hot Encoding
unknown_values = set(input_features_baru_encoded.columns) - set(data.columns)
if unknown_values:
    print("Nilai 'Jenis Sensor' yang tidak dikenal dalam fitur input setelah One-Hot Encoding:", unknown_values)
else:
    # Dapatkan rekomendasi
    hasil_rekomendasi = rekomendasi_kamera(data, model, input_features_baru_encoded)
    print("\nRekomendasi kamera:")
    print(hasil_rekomendasi[['Kamera', 'Brand']].head())


Hasil One-Hot Encoding:
     Price  Color Depth  LowLight ISO  APS-C  Full Frame
0  1000000           24          3200    1.0         0.0
1   750000           30          6400    0.0         1.0
Nilai 'Jenis Sensor' yang tidak dikenal dalam fitur input setelah One-Hot Encoding: {'APS-C', 'Full Frame'}


c:\Users\Laptop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
# 3. Pengujian dan Validasi
# Lakukan pengujian dan validasi untuk memastikan model dan sistem rekomendasi bekerja dengan baik.

# Cross-Validation: Lakukan validasi silang untuk memastikan model stabil

# Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5)
print(f"Skor cross-validation: {cv_scores}")
print(f"Rata-rata skor cross-validation: {cv_scores.mean():.2f}")

Skor cross-validation: [0.63157895 0.68421053 0.73684211 0.78947368 0.72222222]
Rata-rata skor cross-validation: 0.71


In [12]:
# 4. Pengembangan Sistem Pengguna Akhir
# Jika semuanya berjalan dengan baik, langkah terakhir adalah mengembangkan antarmuka pengguna (user interface) untuk sistem pendukung keputusan. Ini bisa dilakukan dengan menggunakan web framework seperti Flask atau Django.

# app = Flask(__name__)

# @app.route('/rekomendasi', methods=['POST'])
# def rekomendasi():
#     data = request.json
#     input_features = pd.DataFrame(data)
#     hasil_rekomendasi = rekomendasi_kamera(data, model, input_features)
#     return jsonify(hasil_rekomendasi.to_dict(orient='records'))

# if __name__ == '__main__':
#     app.run(debug=True)